# forked with minimal edits from Marília Prata https://www.kaggle.com/mpwolke/vacaciones-en-ecuador-with-galearn/notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #7FFFD4;"><b style="color:#00008B;">(Galearn) Genetic Algorithms for model selection by Oliver</b></h1></center>

https://libraries.io/pypi/galearn

Keywords

machine, learning, hyperparameter, tuning, scikit-learn

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4W3KfVgEZEOaF4lbU1AjBz8IOugcF5_7Aow&usqp=CAU)datascienceplus.com

In [ ]:
#Code by Ollibolli https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

!pip install galearn

In [ ]:
#Code by Ollibolli https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

import glob
import os
import gc
from galearn import *
from joblib import Parallel, delayed
from sklearn.model_selection import KFold

from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score
import lightgbm as lgb

In [ ]:
vacaciones = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
vacaciones.head()

<h1><span class="label label-default" style="background-color:#7FFFD4;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#00008B; padding:10px">Holidays and Events</span></h1><br>

Holidays and Events, with metadata

"Pay special attention to the TRANSFERRED COLUMN. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge."

"Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday)."

https://www.kaggle.com/c/store-sales-time-series-forecasting/data

In [ ]:
#Code by EDWIN M MIIRI https://www.kaggle.com/edwincaleb/eda-breast-cancer-classification
#plotting subplots of our target column
f,ax=plt.subplots(1,2,figsize=(10,5))
vacaciones['transferred'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Transferred Holidays')
ax[0].set_ylabel('')
sns.countplot('transferred',data=vacaciones,ax=ax[1])
ax[1].set_title('Transferred Holidays')
plt.show()

In [ ]:
#Codes by https://www.kaggle.com/muhammadismail99/data-analysis-of-guf-pub-dataset/notebook

plt.figure(figsize=(6,4))
sns.countplot(data = vacaciones, x = 'type')
plt.title('Days Types');

In [ ]:
quito = vacaciones[(vacaciones['description']=='Fundacion de Quito')].reset_index(drop=True)
quito.head(6)

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSK1B2IbsT-pIyzhdRMHFf4A2JDxsEsYhy_Vw&usqp=CAU)ecuadoravisos.com

In [ ]:
#Codes by https://www.kaggle.com/muhammadismail99/data-analysis-of-guf-pub-dataset/notebook

sns.countplot(data = vacaciones, x = 'transferred')
plt.title('Transferred Holidays');
#plt.xticks(rotation=45);

In [ ]:
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
train.head()

In [ ]:
#Codes by https://www.kaggle.com/muhammadismail99/data-analysis-of-guf-pub-dataset/notebook

plt.figure(figsize=(6,4))
sns.countplot(data = vacaciones, x = 'locale', palette="GnBu_d",edgecolor="black")
plt.title('Ecuador Holidays');
#plt.xticks(rotation=45);

In [ ]:
train.isnull().sum()

In [ ]:
#Code by Ollibolli https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

path_submissions = '/'

target_name = 'sales'
scores_folds = {}

![](https://imgur.com/Bg0cPix.png)

In [ ]:
#Code by Ollibolli https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

# Function to calculate the root mean squared percentage error
def rmsle(y_true, y_pred):
    return -np.sqrt(np.mean(np.square((np.log(1+y_true) - np.log(1+y_pred)))))

In [ ]:
vacaciones["description"].value_counts()[:10].plot.barh(title='Holidays Description', color='yellow');

In [ ]:
#Code by Ollibolli https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

params = dict()
params['boosting_type'] = ['gbdt', 'dart']
params['n_jobs'] = [-1]
params['num_leaves'] = np.arange(100,700)
params['learning_rate'] = np.linspace(0.0001, 0.5, 1000)
#params['colsample_bytree'] = np.linspace(0.5, 1, 1000)
params['subsample'] = np.linspace(0.25, 0.9, 1000)
params['n_estimators'] = np.arange(50, 500)
params['min_child_weight'] = np.linspace(0.00001, 50, 1000)
#params['min_child_samples'] = np.arange(3, 50)
params['reg_alpha'] = np.linspace(0.001, 1, 1000)
params['reg_lambda'] = np.linspace(0.0001, 0.5, 1000)
params['random_state'] = [42]

In [ ]:
#Code by Ollibolli https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

scorer = make_scorer(rmsle)

In [ ]:
features = [col for col in train.columns if col not in {"date", "sales", "id"}]
y = train['sales']
x = train[features]

In [ ]:
x

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
x['family'] = enc.fit_transform(x['family'])

In [ ]:
#Code by Ollibolli https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

best = simulate(params, scorer, 3,
                        lgb.LGBMRegressor,
                        x, y,
                        selection = 'tournament',
                        pop_size = 15,
                        p_cross = 0.8,
                        p_mutate = 0.5,
                        sim_ann = True)

#Train file does NOT have missing values. Therefore, I don't know what means that Best initial fitness Nan. 

Probably, I did something wrong again. 

In [ ]:
print(best)

In [ ]:
##Code by Taha07  https://www.kaggle.com/taha07/data-scientists-jobs-analysis-visualization/notebook

from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'black',
                      colormap='Set3',
                      height =2000,
                      width = 2000
                     ).generate(str(vacaciones["locale_name"]))
plt.rcParams['figure.figsize'] = (12,12)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Holidays Places")
plt.show()

<h1><span class="label label-default" style="background-color:#7FFFD4;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:#00008B; padding:10px">Acknowledgement</span></h1><br>

Galearn 0.0.1

Oliver (Kaggler Ollibolli) https://www.kaggle.com/ollibolli/lgbm-hyperparameter-search-with-genetic-algorithms

Oliver created a sklearn-API compatible and thus also works with lightgbm. Oliver would be glad to hear your feedback and thoughts. Let him know how it works for you and where he can improves it.

https://pypi.org/project/galearn/